In [ ]:
#Gathering Historical Stock Price Data
import pandas as pd
df=pd.read_csv('MARUTI.csv')
df.head()

df.tail()

df1=df.reset_index()['Close']
df1

import matplotlib.pyplot as plt
plt.plot(df1)

#Scaling Data Between 0 And 1
import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))
df1

#Making Subsets Of Train Data
import numpy as np
num_subsets = 5
subset_percentage = 0.8
subsets = []
for i in range(num_subsets):
    subset_size = int(len(df1) * subset_percentage)
    subset_indices = np.random.choice(len(df1), size=subset_size, replace=False)
    subset_data = df1[subset_indices]
    subsets.append(subset_data)

#Random Sampling Of Test Data
test_data_indices = np.setdiff1d(np.arange(len(df1)), np.arange(len(subsets)))
subset_percentage=0.2
subsets1=[]
for i in range(num_subsets):
    subset_size = int(0.2*len(df1))
    subset_indices = np.random.choice(test_data_indices, size=subset_size, replace=False)
    subset_data = df1[subset_indices]
    subsets1.append(subset_data)

#Making Input Output Split
import numpy as np
def create_dataset(dataset,time_step=1):
  dataX,dataY=[],[]
  for i in range(len(dataset)-time_step-1):
    a=dataset[i:(i+time_step),0]
    dataX.append(a)
    dataY.append(dataset[i+time_step,0])
  return np.array(dataX),np.array(dataY)

#Training Data On SVM
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import math
train_data=subsets[0]
test_data=subsets1[0]
X_train,y_train=create_dataset(train_data,100)
X_test,y_test=create_dataset(test_data,100)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1])
svr_model = SVR(kernel='rbf', C=1.0,gamma='scale')
svr_model.fit(X_train, y_train)
y_pred_train = svr_model.predict(X_train)
y_pred_test = svr_model.predict(X_test)
y_pred1 = y_pred_train.reshape(-1,1)
y_pred2 = y_pred_test.reshape(-1,1)
rmse1 = math.sqrt(mean_squared_error(y_train, y_pred1))
rmse2 = math.sqrt(mean_squared_error(y_test, y_pred2))
print(f'Root Mean Squared Error Train: {rmse1}')
print(f'Root Mean Squared Error Test: {rmse2}')
print(f'MAPE:{np.mean(np.abs((y_test-y_pred2)/y_test))*100}')
from sklearn.metrics import mean_absolute_error
print(f'MAE:{mean_absolute_error(y_test,y_pred2)}')

#Plotting Results
import matplotlib.pyplot as plt
plt.plot(scaler.inverse_transform(y_test.reshape(-1,1)))
plt.plot(scaler.inverse_transform(y_pred2))
plt.title('Actual vs. Predicted Stock Prices')
plt.xlabel('Data Points')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

#LSTM
train_data=subsets[1]
test_data=subsets1[1]
X_train,y_train=create_dataset(train_data,100)
X_test,y_test=create_dataset(test_data,100)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
model = Sequential()
model.add(LSTM(50,input_shape=(100,1)))
model.add(Dense(1,activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

from sklearn.metrics import mean_squared_error
print(f'Train RMSE:{math.sqrt(mean_squared_error(y_train,train_predict))}')
print(f'Test RMSE:{math.sqrt(mean_squared_error(y_test,test_predict))}')
print(f'MAPE:{np.mean(np.abs((y_test-test_predict)/y_test))*100}')
from sklearn.metrics import mean_absolute_error
print(f'MAE:{mean_absolute_error(y_test,test_predict)}')

import matplotlib.pyplot as plt
plt.plot(scaler.inverse_transform(y_test.reshape(-1,1)))
plt.plot(scaler.inverse_transform(test_predict))
plt.title('Actual vs. Predicted Stock Prices')
plt.xlabel('Data Points')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

#Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
train_data=subsets[2]
test_data=subsets1[2]
X_train,y_train=create_dataset(train_data,100)
X_test,y_test=create_dataset(test_data,100)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1])
rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)
y_pred1 = rf_model.predict(X_train)
y_pred2 = rf_model.predict(X_test)
rmse1 = math.sqrt(mean_squared_error(y_train, y_pred1))
rmse2 = math.sqrt(mean_squared_error(y_test, y_pred2))
print(f'Root Mean Squared Error Train: {rmse1}')
print(f'Root Mean Squared Error Test: {rmse2}')
print(f'MAPE:{np.mean(np.abs((y_test-y_pred2)/y_test))*100}')
from sklearn.metrics import mean_absolute_error
print(f'MAE:{mean_absolute_error(y_test,y_pred2)}')
plt.plot(y_test, label='Actual')
plt.plot(y_pred2, label='Predicted')
plt.title('Actual vs. Predicted Stock Prices (Random Forest)')
plt.xlabel('Data Points')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

#Naive Bayes
train_data=subsets[3]
test_data=subsets1[3]
X_train,y_train=create_dataset(train_data,100)
X_test,y_test=create_dataset(test_data,100)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1])
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
bayesian_ridge = BayesianRidge()
bayesian_ridge.fit(X_train, y_train)
y_pred1 = bayesian_ridge.predict(X_train)
y_pred2 = bayesian_ridge.predict(X_test)
rmse1 = math.sqrt(mean_squared_error(y_train, y_pred1))
rmse2 = math.sqrt(mean_squared_error(y_test, y_pred2))
print(f'Mean Squared Error Train: {rmse1}')
print(f'Mean Squared Error Test: {rmse2}')
print(f'MAPE:{np.mean(np.abs((y_test-y_pred2)/y_test))*100}')
from sklearn.metrics import mean_absolute_error
print(f'MAE:{mean_absolute_error(y_test,y_pred2)}')

#KNN
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
train_data=subsets[4]
test_data=subsets1[4]
X_train,y_train=create_dataset(train_data,100)
X_test,y_test=create_dataset(test_data,100)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1])
model1 = KNeighborsRegressor(n_neighbors=5)
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_train)
y_pred2 = model1.predict(X_test)
rmse1 = mean_squared_error(y_train, y_pred1)
rmse2 = mean_squared_error(y_test, y_pred2)
print(f'Mean Squared Error Train: {rmse1}')
print(f'Mean Squared Error Test: {rmse2}')
print(f'MAPE:{np.mean(np.abs((y_test-y_pred2)/y_test))*100}')
from sklearn.metrics import mean_absolute_error
print(f'MAE:{mean_absolute_error(y_test,y_pred2)}')

!pip install scikeras

#Stacking
training_size=int(len(df1)*0.80)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]
X_train,y_train=create_dataset(train_data,100)
X_test,y_test=create_dataset(test_data,100)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1])
from scikeras.wrappers import KerasRegressor
base_models = [('svm', svr_model) ,('lstm', KerasRegressor(model)),('rf', rf_model),('nb', bayesian_ridge), ('knn', model1)]
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
meta_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)
stacking_regressor.fit(X_train, y_train)
y_pred_stacking1 = stacking_regressor.predict(X_train)
y_pred_stacking2 = stacking_regressor.predict(X_test)
rmse_stacking1 = math.sqrt(mean_squared_error(y_train, y_pred_stacking1))
rmse_stacking2 = math.sqrt(mean_squared_error(y_test, y_pred_stacking2))
print(f'Mean Squared Error (Stacking) Train: {rmse_stacking1}')
print(f'Mean Squared Error (Stacking) Test: {rmse_stacking2}')
print(f'MAPE:{np.mean(np.abs((y_test-y_pred_stacking2)/y_test))*100}')
from sklearn.metrics import mean_absolute_error
print(f'MAE:{mean_absolute_error(y_test,y_pred_stacking2)}')

r_squared = stacking_regressor.score(X_test, y_test)
print("Stacking Score:",r_squared)
from sklearn.metrics import r2_score
r2_score_stacking = r2_score(y_test, y_pred_stacking2)
print("Stacking R^2 Score:", r2_score_stacking)

#Plotting Final Data
look_back=100
trainPredictPlot=np.empty_like(df1)
trainPredictPlot[:,:]=np.nan
trainPredictPlot[look_back:len(y_pred_stacking1)+look_back,:]=y_pred_stacking1.reshape(-1,1)
testPredictPlot=np.empty_like(df1)
testPredictPlot[:,:]=np.nan
testPredictPlot[len(y_pred_stacking1)+(look_back*2)+1:len(df1)-1,:]=y_pred_stacking2.reshape(-1,1)
plt.plot(scaler.inverse_transform(df1),color='black')
plt.plot(scaler.inverse_transform(trainPredictPlot),color='red')
plt.plot(scaler.inverse_transform(testPredictPlot),color='red')
plt.legend(['Actual Data', 'Predicted Data'])
plt.xlabel("Day")
plt.ylabel("Stock Price In Rs.")